In [ ]:
import sys
import os

os.chdir("d:/MyProjects/Project-01/Text-Summarizer-Project")

# Add the src directory to PYTHONPATH
sys.path.append(os.path.abspath("src"))

In [ ]:
%pwd

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

    

In [ ]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(str(config_filepath))
        self.params = read_yaml(str(params_filepath))
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
        root_dir=config.root_dir,
        data_path=config.data_path,
        tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config
        
  

In [ ]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [ ]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)


    def convert_examples_to_features(self, example_batch):
        # Tokenize the input and target texts
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

        target_encodings = self.tokenizer(
                    example_batch['summary'], max_length=128, truncation=True)
            
            
        # Return a dictionary with the input and target encodings
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids'],

        }
         
    def convert(self):
        dataset_samsum=load_from_disk(self.config.data_path)
        print(dataset_samsum.column_names)
        dataset_samsum_pt =dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(self.config.root_dir)
        logger.info(f"Dataset saved to {self.config.root_dir}")

        

In [ ]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    logger.exception(f"An error occurred: {e}", exc_info=True)
    # raise e